In [2]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [21]:
drug_arrest = pd.read_csv('/Users/tracy/code/sfpd_data/data/processed/drug_arrest_data.csv')
drug_arrest.groupby('Incident Day of Week').size().sort_values(ascending=False)

Incident Day of Week
Wednesday    3577
Tuesday      3054
Thursday     2879
Friday       2292
Monday       2100
Saturday     1868
Sunday       1668
dtype: int64

In [4]:
alt.__version__

'5.0.0rc1'

In [5]:
drug_arrest.groupby('Incident Category').size().sort_values(ascending=False)

Incident Category
Drug Offense      17195
Drug Violation      243
dtype: int64

In [6]:
drug_arrest.groupby(['Incident Category','Incident Description']).size().sort_values(ascending=False)

Incident Category  Incident Description                     
Drug Offense       Narcotics Paraphernalia, Possession of       4254
                   Methamphetamine, Possession For Sale         1936
                   Cocaine, Base/rock, Possession For Sale      1790
                   Methamphetamine Offense                      1546
                   Heroin, Possession For Sale                  1508
                                                                ... 
                   Barbiturates, Possession For Sale               1
                   Barbiturates, Transportation                    1
                   Methadone, Transportation                       1
                   Methadone, Sale                                 1
                   Controlled Substance, Presence Where Used       1
Length: 62, dtype: int64

In [7]:
drug_arrest.groupby('Analysis Neighborhood').size().sort_values(ascending=False).head(10)

Analysis Neighborhood
Tenderloin                        8613
South of Market                   2962
Mission                           1740
Financial District/South Beach     451
Bayview Hunters Point              437
Castro/Upper Market                291
Western Addition                   273
Haight Ashbury                     242
Nob Hill                           213
North Beach                        137
dtype: int64

In [8]:
drug_arrest.groupby(['Incident Day of Week','Incident Time']).size()

Incident Day of Week  Incident Time
Friday                00:00            20
                      00:01             3
                      00:02             2
                      00:03             1
                      00:07             1
                                       ..
Wednesday             23:50             2
                      23:52             2
                      23:55             2
                      23:56             3
                      23:59             2
Length: 5403, dtype: int64

In [7]:
drug_arrest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17438 entries, 0 to 17437
Data columns (total 35 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            17438 non-null  int64  
 1   Incident Datetime                                     17438 non-null  object 
 2   Incident Date                                         17438 non-null  object 
 3   Incident Time                                         17438 non-null  object 
 4   Incident Year                                         17438 non-null  int64  
 5   Incident Day of Week                                  17438 non-null  object 
 6   Report Datetime                                       17438 non-null  object 
 7   Row ID                                                17438 non-null  int64  
 8   Incident ID                                           17

In [90]:
drug_arrest= drug_arrest.rename(columns={"Incident Date":'incident_date','Row ID':'row_id',
                                         'Incident Day of Week':'day_of_week','Incident Year':'incident_year',
                                         'Incident Time':'incident_time'})
drug_arrest['incident_date'] = pd.to_datetime(drug_arrest['incident_date'])
drug_arrest['incident_time'] = pd.to_datetime(drug_arrest['incident_time'])

counts = drug_arrest.groupby(drug_arrest['incident_date'].dt.strftime('%Y-%m')).size().reset_index(name='count')
counts['incident_date_2'] = counts['incident_date']
counts.head()

,incident_date,count,incident_date_2
0,2018-01,337,2018-01
1,2018-02,356,2018-02
2,2018-03,423,2018-03
3,2018-04,348,2018-04
4,2018-05,384,2018-05


In [50]:
end_date = '2023-02'
filtered_counts = counts[counts['incident_date']<=end_date]
filtered_counts.head()
drug_arrest_median = filtered_counts['count'].median()
drug_arrest_median

271.5

In [84]:
base = alt.Chart(filtered_counts).mark_line(size=4).encode(
    alt.X('incident_date:T',title='Date'),
    alt.Y('count',title='Count'),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Drug-related Arrests, Jan. 2018 - Feb.2023'
)

median = alt.Chart(filtered_counts).mark_rule(size=2,color='crimson').encode(
    y='median(count)',
    tooltip = alt.Tooltip(title = 'Median of counts')
)

base+median

alt.LayerChart(...)

In [91]:
neighborho = drug_arrest.groupby([(drug_arrest['incident_date'].dt.strftime('%Y-%m')),drug_arrest['Analysis Neighborhood']]).size().reset_index(name='count')
neighborho['incident_date_2'] = neighborho['incident_date']
sorted = neighborho.sort_values('count',ascending=True)
sorted


,incident_date,Analysis Neighborhood,count,incident_date_2
704,2020-04,North Beach,1,2020-04
1012,2021-07,Sunset/Parkside,1,2021-07
1276,2022-09,Bernal Heights,1,2022-09
487,2019-07,Pacific Heights,1,2019-07
485,2019-07,Oceanview/Merced/Ingleside,1,2019-07
...,...,...,...,...
746,2020-06,Tenderloin,207,2020-06
251,2018-10,Tenderloin,227,2018-10
1398,2023-02,Tenderloin,267,2023-02
1270,2022-08,Tenderloin,310,2022-08


In [126]:
alt.Chart(sorted).mark_area(size=9,opacity=0.8).encode(
    x= alt.X('incident_date:T',title='Date'),
    y= alt.Y('count:Q',stack = 'normalize',title = 'Count'),
    color=alt.Color("Analysis Neighborhood",legend=alt.Legend(title='Neighborhood', values=['Tenderloin', 'South of Market','Mission'])),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests'),alt.Tooltip('Analysis Neighborhood',title='Neighborhood')]
).properties(
    width=600,
    height=400,
    title='Total Drug Arrests by Neighborhoods (2018-now)'
)

# .transform_filter(
#      alt.FieldEqualPredicate(field='Analysis Neighborhood',equal = 'Tenderloin')
# )

alt.Chart(...)

In [10]:
month_week = drug_arrest.groupby([drug_arrest['incident_date'].dt.strftime('%m'),drug_arrest['day_of_week']]).size().reset_index(name='count')
month_week.head()

,incident_date,day_of_week,count
0,01,Friday,253
1,01,Monday,201
2,01,Saturday,206
3,01,Sunday,206
4,01,Thursday,287


In [11]:
alt.Chart(month_week).mark_rect().encode(
    alt.X('incident_date:O',title = 'Month'),
    alt.Y('day_of_week',title = 'Day of Week'),
    color='count'
)

alt.Chart(...)

In [115]:
week_time = drug_arrest.groupby([drug_arrest['day_of_week'],drug_arrest['incident_time'].dt.strftime('%H')]).size().reset_index(name='count')
week_time['hour']=week_time['incident_time']+':00'
week_time.head()


,day_of_week,incident_time,count,hour
0,Friday,00,73,00:00
1,Friday,01,37,01:00
2,Friday,02,23,02:00
3,Friday,03,33,03:00
4,Friday,04,22,04:00


In [116]:
base = alt.Chart(week_time).mark_rect().encode(
    alt.Y('day_of_week',title = 'Day of Week'),
    alt.X('incident_time',title = 'Hour of Day'),
    color='count',
    tooltip = [alt.Tooltip('day_of_week',title='Day'),
               alt.Tooltip('hour',title = 'Time'),
               alt.Tooltip('incident_time',title='Drug Arrests')]
)
base

alt.Chart(...)

In [106]:
yr_month_week = drug_arrest.groupby([drug_arrest['year'],drug_arrest['incident_date'].dt.strftime('%m'),drug_arrest['day_of_week']]).size().reset_index(name='count')
yr_month_week.head()
alt.Chart(yr_month_week).mark_rect().encode(
    alt.X('incident_date:O',title = 'Month'),
    alt.Y('day_of_week',title = 'Day of Week'),
    color='sum(count)'
)

KeyError: 'year'

In [28]:
"""
base = alt.Chart(counts).mark_bar(size=9).encode(
    alt.X('incident_date:T',title='Date'),
    y='count',
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Incidents by Year-Month'
)"""
experiment = alt.Chart(counts).mark_bar().encode(
    x='incident_date:T',
    y='count')
experiment

alt.Chart(...)

In [36]:
years =[2018,2019,2020,2021,2022,2023]
year_dropdown = alt.binding_select(options=years, name="Year")
year_select = alt.selection_point(fields=['year'], bind=year_dropdown)

filter_years = experiment.add_params(year_select).transform_filter(year_select)

In [37]:
filter_years

alt.Chart(...)

In [24]:
brush = alt.selection_interval()
base = alt.Chart(counts).mark_bar(size=10).encode(
    x='incident_date',
    y='count',
).properties(
    width=600,
    height=400
).add_params(brush)
upper = base.encode(
    alt.X('incident_date:T', scale=alt.Scale(domain=brush)),  
    tooltip = [alt.Tooltip('incident_date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(title='Incidents by Year-Month')

# lower = base.properties(
#     height=60
# ).add_params(brush)

upper.interactive()

alt.Chart(...)

In [26]:
narcan_deploy = pd.read_csv('/Users/tracy/sfpd-data/data/processed/narcan_deployment.csv')


In [39]:
narcan_deploy= narcan_deploy.rename(columns={"Incident Date":'incident_date','Row ID':'row_id',
                                         'Incident Day of Week':'day_of_week','Incident Year':'incident_year',
                                         'Incident Time':'incident_time'})
narcan_deploy['incident_date'] = pd.to_datetime(narcan_deploy['incident_date'])
narcan_deploy['incident_time'] = pd.to_datetime(narcan_deploy['incident_time'])

nd_counts = narcan_deploy.groupby(narcan_deploy['incident_date'].dt.strftime('%Y-%m')).size().reset_index(name='count')
nd_counts['incident_date_2'] = nd_counts['incident_date']
nd_counts.head()
median_nd = nd_counts['count'].median()
median_nd

12.0

In [88]:
end_date = '2023-02'
filtered_nd_counts = nd_counts[nd_counts['incident_date']<=end_date]
median_nd = filtered_nd_counts['count'].median()
base = alt.Chart(filtered_nd_counts).mark_line(size=4).encode(
    alt.X('incident_date:T',title='Date'),
    alt.Y('count',title = 'Counts'),
    tooltip = [alt.Tooltip('incident_date_2',title = 'Date'),alt.Tooltip('count',title='Drug Arrests')]
).properties(
    width=600,
    height=400,
    title='Narcan Deployment, Jan 2018 - Feb 2023'
)
median = alt.Chart(filtered_nd_counts).mark_rule(size=2,color='crimson').encode(
    y='median(count)',
    tooltip = alt.Tooltip(title = 'Median of counts')
)
base+median


alt.LayerChart(...)

In [ ]:
neighborho = drug_arrest.groupby([(drug_arrest['incident_date'].dt.strftime('%Y-%m')),drug_arrest['Analysis Neighborhood']]).size().reset_index(name='count')
neighborho['incident_date_2'] = neighborho['incident_date']
sorted = neighborho.sort_values('count',ascending=True)
sorted